Installed packages in virtual environment

In [2]:
# %pip install pymongo
# %pip install pandas
# %pip install google-cloud-bigquery

In [3]:
import pandas as pd
from pymongo import MongoClient
# from transformers import pipeline

## 1. init connection to database
- Create 2 collections: business, reviews
- upload json files into their respective collections

In [2]:
client = MongoClient()
client = MongoClient('localhost', 27017)

In [3]:
yelp_db = client["Yelp_dataset"] 
business_collection = yelp_db['business'] 
review_collection = yelp_db['reviews']

## 2. Initialise tables

Currently, we need 4 tables
- Business table: { Business_id (PK) , Name , State, City , Stars, Review_count, weighted_review, Categories }
- Names table : {Name (PK) , Number of outlets, Average reviews , Total reviews} 
- Reviews table {Review_id (PK), Business_id (FK), Stars, Text}
- Categories table : {Category, Count (number of appearance), score (avg)}




https://stackoverflow.com/questions/65262832/pre-trained-models-for-text-classification

In [4]:
business_df = []
for row in business_collection.find():
    print(row)
    break
    cleaned_business = {}
    cleaned_business['business_id'] = row['business_id']
    cleaned_business['name'] = row['name']
    cleaned_business['city'] = row['city']
    cleaned_business['state'] = row['state']
    cleaned_business['categories'] = row['categories']
    cleaned_business['review']
    records.append

{'_id': ObjectId('64109c589bc49fe9c1608aeb'), 'business_id': 'Pns2l4eNsfO8kk83dixA6A', 'name': 'Abby Rappoport, LAC, CMQ', 'address': '1616 Chapala St, Ste 2', 'city': 'Santa Barbara', 'state': 'CA', 'postal_code': '93101', 'latitude': 34.4266787, 'longitude': -119.7111968, 'stars': 5, 'review_count': 7, 'is_open': 0, 'attributes': {'ByAppointmentOnly': 'True'}, 'categories': 'Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists', 'hours': None}


In [21]:
# Output for names table
names_record = []
for j in business_collection.aggregate([{'$group' : {'_id' : '$name', 'average_review' : {'$avg' : '$stars'}, 'total_reviews' : {'$sum' : '$review_count'},
                                                      'number of outlets' : {'$sum' : 1}}},{'$sort' : {'number of outlets' : -1}}]):
    names_record.append(j)

In [22]:
names_df = pd.DataFrame(
    names_record,
    # In the loaded table, the column order reflects the order of the
    # columns in the DataFrame.
    columns=[
        "_id",
        "average_review",
        "total_reviews",
        "number of outlets",
    ])
names_df

,_id,average_review,total_reviews,number of outlets
0,Starbucks,3.126381,20692,724
1,McDonald's,1.863442,17359,703
2,Dunkin',2.302941,9864,510
3,Subway,2.586057,4123,459
4,Taco Bell,2.154795,8325,365
...,...,...,...,...
114112,Artisano's Oils & Spices,5.000000,38,1
114113,Twin Oaks Park,5.000000,5,1
114114,Palm Shadows Apartments,2.000000,6,1
114115,Stor-Mor Sheds,4.500000,13,1


## 3. Load data into warehouse
https://cloud.google.com/bigquery/docs/samples/bigquery-load-table-dataframe 

In [6]:
from google.cloud import bigquery